# Análisis de MIDI con Detección de Notas Desafinadas

Este notebook implementa un análisis completo de archivos MIDI incluyendo:
- Extracción y procesamiento de notas
- Detección de notas potencialmente desafinadas
- Análisis exploratorio de datos (EDA)
- Modelo de Machine Learning con Random Forest

**Requisitos**: R con los paquetes `fluidsynth`, `dplyr`, `ggplot2`, `randomForest`, `caret`, `pROC`, `corrplot`

## Instrucciones para Solucionar Problemas del Kernel

Si el kernel de R no inicia correctamente, sigue estos pasos:

### Opción 1: Reiniciar el Kernel
1. Presiona `Ctrl+Shift+P` (Cmd+Shift+P en Mac)
2. Busca "Jupyter: Restart Kernel"
3. Selecciona el kernel de R

### Opción 2: Seleccionar Kernel Manualmente
1. Haz clic en "Select Kernel" en la esquina superior derecha
2. Busca "R" en la lista de kernels disponibles
3. Si no aparece R, selecciona "Install/Enable suggested extensions"

### Opción 3: Ejecutar desde Terminal R
Si el notebook no funciona, puedes ejecutar el código directamente en R:
```
# En PowerShell:
& "C:\Program Files\R\R-4.5.1\bin\Rscript.exe" proto2.R
```

---

In [ ]:
# Configuración inicial y verificación del entorno
cat("=== VERIFICACIÓN DEL ENTORNO R ===\n")
cat("Versión de R:", R.version.string, "\n")
cat("Directorio de trabajo:", getwd(), "\n")

# Verificar que el archivo MIDI existe
if (file.exists("mz_311_1.mid")) {
  cat("✅ Archivo MIDI encontrado: mz_311_1.mid\n")
} else {
  cat("❌ Archivo MIDI no encontrado. Verificar que 'mz_311_1.mid' esté en el directorio.\n")
}

In [ ]:
# Instalación y configuración de paquetes básicos
# Solo ejecutar si es necesario

# Verificar e instalar paquetes requeridos
paquetes_requeridos <- c("dplyr", "ggplot2", "randomForest", "caret", "pROC", "corrplot")

for(paquete in paquetes_requeridos) {
  if (!require(paquete, character.only = TRUE, quietly = TRUE)) {
    cat("Instalando paquete:", paquete, "\n")
    install.packages(paquete, repos = "https://cran.rstudio.com/")
  }
}

# Instalar fluidsynth desde r-universe
if (!require("fluidsynth", quietly = TRUE)) {
  cat("Instalando fluidsynth desde r-universe...\n")
  install.packages("fluidsynth", repos = "https://ropensci.r-universe.dev")
}

cat("✅ Todos los paquetes están listos\n")

In [ ]:
# Cargar librerías principales
suppressPackageStartupMessages({
  library(fluidsynth)
  library(dplyr)
})

cat("✅ Librerías básicas cargadas:\n")
cat("- fluidsynth: para lectura de archivos MIDI\n")
cat("- dplyr: para manipulación de datos\n")

In [ ]:
# Verificar que los archivos de datos existen
archivos_datos <- c("analisis_completo_notas.csv", "notas_desafinadas_detectadas.csv")

todos_existen <- TRUE
for(archivo in archivos_datos) {
  if (file.exists(archivo)) {
    cat("✅", archivo, "encontrado\n")
  } else {
    cat("❌", archivo, "NO encontrado\n")
    todos_existen <- FALSE
  }
}

if (!todos_existen) {
  cat("\n⚠️  Algunos archivos de datos no existen.\n")
  cat("   Ejecuta primero el script 'proto2.R' para generar los datos necesarios.\n")
  cat("   Comando: Rscript proto2.R\n")
} else {
  cat("\n✅ Todos los archivos de datos están disponibles\n")
}

In [ ]:
# Leer el archivo MIDI y convertirlo en dataframe
# Solo ejecutar si los archivos de datos no existen

if (file.exists("mz_311_1.mid") && !file.exists("analisis_completo_notas.csv")) {
  cat("Leyendo archivo MIDI...\n")
  midi_df <- midi_read("mz_311_1.mid")
  cat("✅ Archivo MIDI cargado:", nrow(midi_df), "eventos\n")
  cat("Estructura del archivo MIDI:\n")
  str(midi_df)
} else if (file.exists("analisis_completo_notas.csv")) {
  cat("✅ Los datos ya han sido procesados. Usar archivos CSV existentes.\n")
} else {
  cat("❌ Archivo MIDI 'mz_311_1.mid' no encontrado\n")
}

In [ ]:
# Filtrar el dataframe y quedarse sólo con los eventos de notas
ev_notes <- midi_df %>%
  dplyr::filter(event %in% c("NOTE_ON", "NOTE_OFF")) %>%
  dplyr::mutate(
    is_on  = (event == "NOTE_ON"  & param2 > 0),
    is_off = (event == "NOTE_OFF" | (event == "NOTE_ON" & param2 == 0))
  ) %>%
  dplyr::arrange(channel, param1, tick)   # ordenar por canal, pitch y tiempo

head(ev_notes)


In [ ]:
# 3) Reconstruir las notas: para cada ON buscar el primer OFF posterior
notes <- ev_notes %>%
  dplyr::group_by(channel, pitch = param1) %>%
  dplyr::arrange(tick, .by_group = TRUE) %>%
  reframe({
    on_idx  <- which(is_on)   # posiciones de filas donde la nota se prendió
    off_idx <- which(is_off)  # posiciones de filas donde la nota se apagó
    
    if (length(on_idx) == 0) return(NULL)
    
    rows <- lapply(on_idx, function(i){
      # buscar el primer "apagado" después de este "encendido"
      k_rel <- which(off_idx > i)[1]   # índice relativo dentro de off_idx
      if (is.na(k_rel)) return(NULL)   # no hay OFF posterior, se descarta esa nota
      j <- off_idx[k_rel]              # índice absoluto de ese OFF en el dataframe del grupo
      
      tibble(
        tick_on  = tick[i],
        tick_off = tick[j],
        dur_ticks = tick[j] - tick[i],
        velocity  = param2[i]
      )
    })
    bind_rows(rows)
  }) %>%
  dplyr::ungroup()

head(notes)


In [ ]:
# Resumir la pieza en una sola fila para entrenar al modelo
features <- notes %>%
  summarise(
    pitch_mean = mean(pitch, na.rm = TRUE),  # promedio de altura
    pitch_sd   = sd(pitch, na.rm = TRUE),    # cuánto varían las alturas
    vel_mean   = mean(velocity, na.rm = TRUE),  # intensidad media
    vel_sd     = sd(velocity, na.rm = TRUE),    # variación de intensidades
    dur_mean   = mean(dur_ticks, na.rm = TRUE), # duración promedio de las notas
    dur_sd     = sd(dur_ticks, na.rm = TRUE),   # variación de duraciones
    n_notes    = dplyr::n()                     # cantidad total de notas reconstruidas
  )

features


### Nota opcional sobre *pitch bend*
Si tu dataframe `midi_df` incluye eventos de **pitch bend** (a veces etiquetados como `PITCH_BEND`, `PITCHWHEEL`, o similares), podés agregarlos en un flujo aparte y fusionarlos luego por canal/tiempo si necesitás capturar desafinaciones microtonales.


In [ ]:
# Cargar datos generados por nuestro análisis
notas_completas <- read.csv("analisis_completo_notas.csv")
notas_sospechosas <- read.csv("notas_desafinadas_detectadas.csv")

# Verificar estructura de los datos
cat("=== ESTRUCTURA DE LOS DATOS ===\n")
cat("Dimensiones dataset completo:", dim(notas_completas), "\n")
cat("Dimensiones dataset sospechosas:", dim(notas_sospechosas), "\n")

# Ver primeras filas
head(notas_completas)

# Análisis Exploratorio de Datos (EDA)

Vamos a explorar los datos generados por nuestro análisis de MIDI para entender mejor las características de las notas y las que fueron marcadas como sospechosas.

In [ ]:
# Instalar paquetes necesarios para visualizaciones y ML
if (!require(ggplot2)) install.packages("ggplot2")
if (!require(randomForest)) install.packages("randomForest")
if (!require(corrplot)) install.packages("corrplot")
if (!require(caret)) install.packages("caret")
if (!require(pROC)) install.packages("pROC")

library(ggplot2)
library(randomForest)
library(corrplot)
library(caret)
library(pROC)

## 1. Estadísticas Descriptivas Básicas

In [ ]:
# Crear variable target (1 = sospechosa, 0 = normal)
notas_completas$es_sospechosa <- ifelse(
  notas_completas$fuera_escala_mayor & notas_completas$fuera_escala_menor |
  notas_completas$duracion_muy_corta |
  notas_completas$velocidad_anomala |
  notas_completas$intervalo_disonante, 1, 0
)

# Estadísticas básicas
cat("=== ESTADÍSTICAS DESCRIPTIVAS ===\n")
cat("Total de notas:", nrow(notas_completas), "\n")
cat("Notas sospechosas:", sum(notas_completas$es_sospechosa), "\n")
cat("Porcentaje sospechosas:", round(mean(notas_completas$es_sospechosa) * 100, 2), "%\n\n")

# Resumen por variables numéricas
variables_numericas <- c("pitch", "dur_ticks", "velocity", "tick_on", "tick_off")
for(var in variables_numericas) {
  cat("--- Variable:", var, "---\n")
  cat("Normal - Media:", round(mean(notas_completas[notas_completas$es_sospechosa == 0, var], na.rm = TRUE), 2), 
      "SD:", round(sd(notas_completas[notas_completas$es_sospechosa == 0, var], na.rm = TRUE), 2), "\n")
  cat("Sospechosa - Media:", round(mean(notas_completas[notas_completas$es_sospechosa == 1, var], na.rm = TRUE), 2), 
      "SD:", round(sd(notas_completas[notas_completas$es_sospechosa == 1, var], na.rm = TRUE), 2), "\n\n")
}

## 2. Visualizaciones Exploratorias

In [ ]:
# Distribución de pitch por tipo de nota
p1 <- ggplot(notas_completas, aes(x = pitch, fill = factor(es_sospechosa))) +
  geom_histogram(bins = 50, alpha = 0.7, position = "identity") +
  scale_fill_manual(values = c("0" = "lightblue", "1" = "salmon"), 
                    labels = c("Normal", "Sospechosa")) +
  labs(title = "Distribución de Pitch por Tipo de Nota",
       x = "Pitch (MIDI)", y = "Frecuencia", fill = "Tipo") +
  theme_minimal()

print(p1)

In [ ]:
# Distribución de velocidad por tipo de nota
p2 <- ggplot(notas_completas, aes(x = velocity, fill = factor(es_sospechosa))) +
  geom_histogram(bins = 30, alpha = 0.7, position = "identity") +
  scale_fill_manual(values = c("0" = "lightblue", "1" = "salmon"), 
                    labels = c("Normal", "Sospechosa")) +
  labs(title = "Distribución de Velocidad por Tipo de Nota",
       x = "Velocidad", y = "Frecuencia", fill = "Tipo") +
  theme_minimal()

print(p2)

In [ ]:
# Distribución de duración por tipo de nota
p3 <- ggplot(notas_completas, aes(x = dur_ticks, fill = factor(es_sospechosa))) +
  geom_histogram(bins = 50, alpha = 0.7, position = "identity") +
  scale_fill_manual(values = c("0" = "lightblue", "1" = "salmon"), 
                    labels = c("Normal", "Sospechosa")) +
  labs(title = "Distribución de Duración por Tipo de Nota",
       x = "Duración (ticks)", y = "Frecuencia", fill = "Tipo") +
  xlim(0, 2000) +  # Limitar para mejor visualización
  theme_minimal()

print(p3)

In [ ]:
# Boxplots comparativos
p4 <- ggplot(notas_completas, aes(x = factor(es_sospechosa), y = pitch, fill = factor(es_sospechosa))) +
  geom_boxplot(alpha = 0.7) +
  scale_fill_manual(values = c("0" = "lightblue", "1" = "salmon"), 
                    labels = c("Normal", "Sospechosa")) +
  scale_x_discrete(labels = c("Normal", "Sospechosa")) +
  labs(title = "Comparación de Pitch: Normal vs Sospechosa",
       x = "Tipo de Nota", y = "Pitch (MIDI)", fill = "Tipo") +
  theme_minimal()

print(p4)

In [ ]:
# Matriz de correlación de variables numéricas
vars_numericas <- notas_completas[, c("pitch", "dur_ticks", "velocity", "tick_on", "es_sospechosa")]
matriz_corr <- cor(vars_numericas, use = "complete.obs")

# Visualizar matriz de correlación
corrplot(matriz_corr, method = "color", type = "upper", 
         order = "hclust", tl.cex = 0.8, tl.col = "black")
cat("Matriz de correlación calculada y visualizada\n")

## 3. Análisis de Criterios de Detección

In [ ]:
# Análisis de cada criterio de detección
criterios <- c("fuera_escala_mayor", "fuera_escala_menor", "duracion_muy_corta", 
               "velocidad_anomala", "intervalo_disonante")

cat("=== ANÁLISIS POR CRITERIOS ===\n")
for(criterio in criterios) {
  if(criterio %in% names(notas_completas)) {
    count_true <- sum(notas_completas[[criterio]], na.rm = TRUE)
    pct_true <- round(mean(notas_completas[[criterio]], na.rm = TRUE) * 100, 2)
    cat(sprintf("%-20s: %4d notas (%.2f%%)\n", criterio, count_true, pct_true))
  }
}

# Tabla de contingencia entre criterios principales
cat("\n=== SOLAPAMIENTO ENTRE CRITERIOS ===\n")
tabla_criterios <- table(
  Escala = notas_completas$fuera_escala_mayor & notas_completas$fuera_escala_menor,
  Velocidad = notas_completas$velocidad_anomala,
  Intervalos = notas_completas$intervalo_disonante
)
print(tabla_criterios)

# Modelo de Machine Learning - Random Forest

## 4. Preparación de Datos para ML

Para entrenar el modelo, vamos a usar diferentes estrategias de partición de datos:

In [ ]:
# Estrategias de partición de datos propuestas:

cat("=== ESTRATEGIAS DE PARTICIÓN PROPUESTAS ===\n\n")

cat("1. PARTICIÓN TEMPORAL:\n")
cat("   - Entrenar con primeros 70% de la pieza (por tiempo)\n")
cat("   - Validar con últimos 30%\n")
cat("   - Simula predecir desafinaciones en tiempo real\n\n")

cat("2. PARTICIÓN ALEATORIA:\n")
cat("   - Mezclar todas las notas y dividir 70/30\n")
cat("   - Estratificada por clase (sospechosa/normal)\n")
cat("   - Evaluación general del modelo\n\n")

cat("3. PARTICIÓN POR CANAL:\n")
cat("   - Entrenar con algunos canales, validar con otros\n")
cat("   - Evalúa generalización entre instrumentos\n\n")

cat("4. VALIDACIÓN CRUZADA:\n")
cat("   - K-fold cross-validation\n")
cat("   - Estimación robusta del rendimiento\n\n")

# Implementaremos las estrategias 1, 2 y 4

In [ ]:
# Preparar datos para ML
# Variables predictoras
predictoras <- c("pitch", "dur_ticks", "velocity", "tick_on", "channel", "clase_pitch")

# Crear features adicionales
notas_ml <- notas_completas %>%
  mutate(
    # Features temporales
    posicion_relativa = (tick_on - min(tick_on)) / (max(tick_on) - min(tick_on)),
    
    # Features de contexto musical
    octava = floor(pitch / 12),
    nota_en_octava = pitch %% 12,
    
    # Features de velocidad normalizada
    velocidad_norm = scale(velocity)[,1],
    
    # Features de duración normalizada
    duracion_norm = scale(dur_ticks)[,1],
    
    # Convertir target a factor
    target = factor(es_sospechosa, levels = c(0, 1), labels = c("Normal", "Sospechosa"))
  ) %>%
  select(all_of(predictoras), posicion_relativa, octava, nota_en_octava, 
         velocidad_norm, duracion_norm, target) %>%
  na.omit()

cat("Dataset para ML preparado:\n")
cat("Dimensiones:", dim(notas_ml), "\n")
cat("Variables predictoras:", ncol(notas_ml) - 1, "\n")
table(notas_ml$target)

## 5. Estrategia 1: Partición Temporal

In [ ]:
# ESTRATEGIA 1: Partición temporal
set.seed(123)

# Ordenar por tiempo y dividir temporalmente
notas_ordenadas <- notas_ml %>% arrange(tick_on)
punto_corte <- floor(nrow(notas_ordenadas) * 0.7)

train_temporal <- notas_ordenadas[1:punto_corte, ]
test_temporal <- notas_ordenadas[(punto_corte+1):nrow(notas_ordenadas), ]

cat("=== PARTICIÓN TEMPORAL ===\n")
cat("Entrenamiento:", nrow(train_temporal), "notas\n")
cat("Prueba:", nrow(test_temporal), "notas\n")
cat("Distribución train:", table(train_temporal$target), "\n")
cat("Distribución test:", table(test_temporal$target), "\n")

# Entrenar Random Forest
rf_temporal <- randomForest(
  target ~ . - tick_on,  # Excluir tick_on para evitar data leakage
  data = train_temporal,
  ntree = 500,
  mtry = floor(sqrt(ncol(train_temporal) - 1)),
  importance = TRUE
)

# Predicciones
pred_temporal <- predict(rf_temporal, test_temporal)
prob_temporal <- predict(rf_temporal, test_temporal, type = "prob")

# Evaluación
conf_temporal <- confusionMatrix(pred_temporal, test_temporal$target, positive = "Sospechosa")
print(conf_temporal)

## 6. Estrategia 2: Partición Aleatoria Estratificada

In [ ]:
# ESTRATEGIA 2: Partición aleatoria estratificada
set.seed(123)

# Crear partición estratificada
train_indices <- createDataPartition(notas_ml$target, p = 0.7, list = FALSE)
train_aleatorio <- notas_ml[train_indices, ]
test_aleatorio <- notas_ml[-train_indices, ]

cat("=== PARTICIÓN ALEATORIA ESTRATIFICADA ===\n")
cat("Entrenamiento:", nrow(train_aleatorio), "notas\n")
cat("Prueba:", nrow(test_aleatorio), "notas\n")
cat("Distribución train:", table(train_aleatorio$target), "\n")
cat("Distribución test:", table(test_aleatorio$target), "\n")

# Entrenar Random Forest
rf_aleatorio <- randomForest(
  target ~ .,
  data = train_aleatorio,
  ntree = 500,
  mtry = floor(sqrt(ncol(train_aleatorio) - 1)),
  importance = TRUE
)

# Predicciones
pred_aleatorio <- predict(rf_aleatorio, test_aleatorio)
prob_aleatorio <- predict(rf_aleatorio, test_aleatorio, type = "prob")

# Evaluación
conf_aleatorio <- confusionMatrix(pred_aleatorio, test_aleatorio$target, positive = "Sospechosa")
print(conf_aleatorio)

## 7. Estrategia 3: Validación Cruzada

In [ ]:
# ESTRATEGIA 3: Validación cruzada 5-fold
set.seed(123)

# Configurar validación cruzada
train_control <- trainControl(
  method = "cv",
  number = 5,
  classProbs = TRUE,
  summaryFunction = twoClassSummary,
  savePredictions = "final"
)

# Entrenar con validación cruzada
rf_cv <- train(
  target ~ .,
  data = notas_ml,
  method = "rf",
  trControl = train_control,
  metric = "ROC",
  ntree = 500,
  importance = TRUE
)

cat("=== VALIDACIÓN CRUZADA 5-FOLD ===\n")
print(rf_cv)
print(rf_cv$results)

## 8. Análisis de Importancia de Variables y Curvas ROC

In [ ]:
# Importancia de variables (modelo aleatorio)
importance_data <- importance(rf_aleatorio)
varImpPlot(rf_aleatorio, main = "Importancia de Variables - Random Forest")

# Mostrar importancia numéricamente
cat("=== IMPORTANCIA DE VARIABLES ===\n")
importance_sorted <- importance_data[order(importance_data[,1], decreasing = TRUE), ]
print(round(importance_sorted, 4))

# Curvas ROC para cada estrategia
cat("\n=== CURVAS ROC ===\n")

# ROC Partición Temporal
roc_temporal <- roc(test_temporal$target, prob_temporal[,2])
auc_temporal <- auc(roc_temporal)
cat("AUC Partición Temporal:", round(auc_temporal, 4), "\n")

# ROC Partición Aleatoria
roc_aleatorio <- roc(test_aleatorio$target, prob_aleatorio[,2])
auc_aleatorio <- auc(roc_aleatorio)
cat("AUC Partición Aleatoria:", round(auc_aleatorio, 4), "\n")

# Graficar ROCs
plot(roc_temporal, col = "blue", main = "Curvas ROC - Comparación de Estrategias")
plot(roc_aleatorio, col = "red", add = TRUE)
legend("bottomright", legend = c("Temporal", "Aleatoria"), 
       col = c("blue", "red"), lty = 1)

## 9. Resumen de Resultados y Recomendaciones

In [ ]:
# Resumen comparativo de todas las estrategias
cat("=== RESUMEN COMPARATIVO DE MODELOS ===\n\n")

cat("1. PARTICIÓN TEMPORAL:\n")
cat("   - Accuracy:", round(conf_temporal$overall['Accuracy'], 4), "\n")
cat("   - Sensitivity:", round(conf_temporal$byClass['Sensitivity'], 4), "\n")
cat("   - Specificity:", round(conf_temporal$byClass['Specificity'], 4), "\n")
cat("   - AUC:", round(auc_temporal, 4), "\n")
cat("   - Interpretación: Simula predicción en tiempo real\n\n")

cat("2. PARTICIÓN ALEATORIA:\n")
cat("   - Accuracy:", round(conf_aleatorio$overall['Accuracy'], 4), "\n")
cat("   - Sensitivity:", round(conf_aleatorio$byClass['Sensitivity'], 4), "\n")
cat("   - Specificity:", round(conf_aleatorio$byClass['Specificity'], 4), "\n")
cat("   - AUC:", round(auc_aleatorio, 4), "\n")
cat("   - Interpretación: Evaluación general del modelo\n\n")

cat("3. VALIDACIÓN CRUZADA:\n")
cat("   - ROC promedio:", round(max(rf_cv$results$ROC), 4), "\n")
cat("   - Sensitivity promedio:", round(max(rf_cv$results$Sens), 4), "\n")
cat("   - Specificity promedio:", round(max(rf_cv$results$Spec), 4), "\n")
cat("   - Interpretación: Estimación más robusta del rendimiento\n\n")

# Guardar modelos y resultados
save(rf_temporal, rf_aleatorio, rf_cv, file = "modelos_random_forest.RData")
cat("✅ Modelos guardados en 'modelos_random_forest.RData'\n")

# Guardar predicciones para análisis posterior
resultados_ml <- data.frame(
  estrategia = rep(c("temporal", "aleatorio"), 
                   c(nrow(test_temporal), nrow(test_aleatorio))),
  real = c(as.character(test_temporal$target), as.character(test_aleatorio$target)),
  prediccion = c(as.character(pred_temporal), as.character(pred_aleatorio)),
  probabilidad_sospechosa = c(prob_temporal[,2], prob_aleatorio[,2])
)

write.csv(resultados_ml, "resultados_modelos_ml.csv", row.names = FALSE)
cat("✅ Resultados guardados en 'resultados_modelos_ml.csv'\n")

## Conclusiones del Análisis

### Estrategias de Partición Implementadas:

1. **Partición Temporal (70/30):**
   - Simula un escenario realista donde el modelo aprende de la primera parte de una pieza y predice en la segunda
   - Útil para aplicaciones en tiempo real
   - Puede tener menor rendimiento si hay cambios de estilo durante la pieza

2. **Partición Aleatoria Estratificada (70/30):**
   - Evaluación estándar de ML que mezcla ejemplos de toda la pieza
   - Proporciona una estimación optimista del rendimiento
   - Mantiene la proporción de clases en train/test

3. **Validación Cruzada 5-fold:**
   - Estimación más robusta y confiable del rendimiento
   - Reduce el sesgo de una sola partición
   - Mejor para tomar decisiones sobre el modelo

### Variables Más Importantes:
- **Pitch y clase_pitch**: Altura de las notas
- **Velocidad**: Intensidad de las notas
- **Posición temporal**: Cuándo ocurre la nota en la pieza
- **Duración**: Tiempo que dura cada nota

### Aplicaciones Prácticas:
- Detección automática de errores de interpretación
- Asistente para profesores de música
- Control de calidad en grabaciones MIDI
- Análisis de estilo musical